In [1]:
import pandas as pd
import numpy as np

# Load data from CSV file in the folder
df = pd.read_csv('downsample.csv')
print(f"{len(list(df.columns))} columns loaded")
######################################
# Data Cleaning
threshold = 0.8
missing_ratio = df.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > threshold].index
df = df.drop(columns=cols_to_drop)

# For those columns with mixed data types, keep only those that are essential
mixed_type_cols = [col for col in df.columns if df[col].apply(type).nunique() > 1]
keep_cols=['Origin State', 'Origin Postal Code', 'Destination Postal Code',
       'What Failed Code', 'How Failed Code', 'Failure Cause Code',
       'Contact State', 'Contact Postal Code']
df = df.drop(columns=[col for col in mixed_type_cols if col not in keep_cols])

# Remove columns with a single unique value
single_value_cols = [col for col in df.columns if df[col].nunique(dropna=False) == 1]
df = df.drop(columns=single_value_cols)
#########################################
## after manual review, drop these columns
drop_columns = ["Report Submission Source","Multiple Rows Per Incident","Carrier Reporter Name","Shipper Name","Origin Country","Undeclared Hazmat Shipment Ind","Contact Name","Contact Title"]
df = df.drop(columns=drop_columns)
#########################################
print(f"{len(list(df.columns))} columns remaining after cleaning")
X = df.drop(columns=['Total Amount Of Damages'])
y = df['Total Amount Of Damages']

177 columns loaded
97 columns remaining after cleaning


In [2]:
X

,Report Number,Date Of Incident,Time Of Incident,Incident City,Incident County,Incident State,Mode Of Transportation,Transportation Phase,Carrier Reporter Street Name,Carrier Reporter City,...,Cont1 Pkg Number In Shipment,Total Hazmat Hosp Injuries,Hazmat Nonhosp Employees.1,Employees Evacuated,Undeclared Shipment,Hmis Serious Fatality,Hmis Serious Injury,Hmis Serious Evacuations,Hmis Serious Major Artery,Hmis Serious Bulk Release
0,<a href = https://portal.phmsa.dot.gov/PDFGene...,12/15/14,750.0,OKLAHOMA CITY,OKLAHOMA,OK,Highway,In Transit,415 N Plainview RD,Ardmore,...,1.0,0.0,0.0,0.0,Yes,No,No,No,No,Yes
1,<a href = https://portal.phmsa.dot.gov/PDFGene...,11/10/10,1408.0,MOUNT JULIET,WILSON,TN,Highway,In Transit,P.O. Box 375,NEWBURY,...,1.0,0.0,0.0,0.0,No,No,No,No,Yes,Yes
2,I-2006010669,12/8/05,337.0,LORDSBURG,HIDALGO,NM,Rail,In Transit,1400 DOUGLAS ST MAIL STOP 1040,OMAHA,...,1.0,0.0,0.0,0.0,No,No,No,No,No,Yes
3,<a href = https://portal.phmsa.dot.gov/PDFGene...,4/28/06,1555.0,LINDEN,MARENGO,AL,Rail,In Transit,136 N. Mt. Pleasant Avenue,MONROEVILLE,...,1.0,0.0,0.0,0.0,No,No,No,No,No,Yes
4,I-2005030104,1/8/05,1542.0,SEABROOK,HARRIS,TX,Rail,In Transit,1400 DOUGLAS ST MAIL STOP 1040,OMAHA,...,1.0,0.0,0.0,0.0,No,No,No,No,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2638,<a href = https://portal.phmsa.dot.gov/PDFGene...,11/18/03,500.0,EGAN,ACADIA,LA,Highway,In Transit,1051 OLD WARRIOR RIVER ROAD,DOLOMITE,...,1.0,0.0,0.0,0.0,No,No,No,No,No,No
2639,I-1996050309,4/15/96,1330.0,LUXEMBURG,DUBUQUE,IA,Highway,In Transit,5500 CENEX DR,INVER GROVE HEIGHTS,...,1.0,0.0,0.0,0.0,No,Yes,No,Yes,No,Yes
2640,<a href = https://portal.phmsa.dot.gov/PDFGene...,5/20/95,530.0,HARMONY,FILLMORE,MN,Highway,In Transit,PO BOX 126 40925 403RD AVE,SAUK CENTRE,...,1.0,0.0,0.0,0.0,No,No,No,No,No,Yes
2641,<a href = https://portal.phmsa.dot.gov/PDFGene...,4/12/95,2330.0,SPARTANBURG,SPARTANBURG,SC,Highway,Loading,422 SOUTH CHURCH ST,CHARLOTTE,...,1.0,0.0,0.0,0.0,No,No,No,No,No,Yes


In [4]:
mixed_type_columns = [col for col in X.columns if X[col].apply(type).nunique() > 1]
for col in mixed_type_columns:
    type_counts = X[col].apply(type).value_counts(normalize=True)
    print(f"Column: {col}")
    for t, pct in type_counts.items():
        print(f"  Type: {t.__name__}, Percentage: {pct:.2%}")
        examples = X[col][X[col].apply(type) == t].dropna().unique()[:5]
        print(f"    Examples: {examples}")
    print()

Column: Origin State
  Type: float, Percentage: 57.55%
    Examples: []
  Type: str, Percentage: 42.45%
    Examples: ['NM' 'MS' 'TX' 'CA' 'MT']

Column: Origin Postal Code
  Type: float, Percentage: 69.01%
    Examples: []
  Type: str, Percentage: 30.99%
    Examples: ['88043' '79720' '93230-9591' '59714-9145' '44004-3943']

Column: Destination Postal Code
  Type: str, Percentage: 72.38%
    Examples: ['74361' '38109' '85235' '36916' '55917']
  Type: float, Percentage: 27.62%
    Examples: []

Column: What Failed Code
  Type: str, Percentage: 69.96%
    Examples: ['161' '137' '140' '104' '150']
  Type: float, Percentage: 30.04%
    Examples: []

Column: How Failed Code
  Type: str, Percentage: 70.22%
    Examples: ['304' '312' '311' '310' '308']
  Type: float, Percentage: 29.78%
    Examples: []

Column: Failure Cause Code
  Type: str, Percentage: 85.47%
    Examples: ['510' '537' '509' '508' '512']
  Type: float, Percentage: 14.53%
    Examples: []

Column: Contact State
  Type: str,